# 주요 세그멘테이션 모델 정리

## 3. 주요 세그멘테이션 모델 (1) FCN

> __*주요 참고 자료*__
>
>* _[Fully Convolutional Networks for Semantic Segmentation - 허다운](https://www.youtube.com/watch?v=_52dopGu3Cw&feature=youtu.be&ab_channel=%EB%94%A5%EB%9F%AC%EB%8B%9D%EB%85%BC%EB%AC%B8%EC%9D%BD%EA%B8%B0%EB%AA%A8%EC%9E%84)_
>* _[FCN 논문 리뷰 — Fully Convolutional Networks for Semantic Segmentation](https://medium.com/@msmapark2/fcn-%EB%85%BC%EB%AC%B8-%EB%A6%AC%EB%B7%B0-fully-convolutional-networks-for-semantic-segmentation-81f016d76204)_
>* _원본 논문: [Fully Convolutional Networks for Semantic Segmentation](https://arxiv.org/abs/1411.4038)_

<img src="./image/fcn.png" />

* FCN은 이미지넷 챌린지(ImageNet Challenge)에서 좋은 성적을 거두었던 AlexNet, VGG-16 등의 모델을 세그멘테이션에 맞게 변형한 모델이다.
* FCN에서는 세그멘테이션을 하기 위해서 네트워크 뒷단에 fully connected layer 대신 CNN을 붙여준다.

<img src="./image/fcn2.png" />

여기서 fully connected layer 대신 CNN을 붙여주는 이유는 CNN은 이미지 내 위치의 특성을 유지하는데, fully connected layer는 위치를 고려하지 않는다. 따라서, 위치정보를 유지하면서 클래스 단위의 히트맵(heat map)을 얻어 세그멘테이션을 하기 위해서, fully connected layer를 CNN으로 대체하는 것이다.

위치의 특성을 유지하면서 이미지 분류를 하기 위해서 마지막 CNN은 __1x1의 커널 크기(kernel size)와 클래스의 개수만큼의 채널__을 갖는다. 클래스 히트맵을 얻기 위해서이다.

CNN과 pooling 레이어를 거치면서 히트맵 크기가 줄어든다, 이를 키워주기 위해서 __upsampling__을 쓴다. Upsampling에는 여러 가지 방법이 있다. 그중 FCN에서는 Deconvolution과 Interpolation 방식을 활용한다. 

* 1차원 상의 두 개의 점 사이에서 거리 비에 따라 추정하는 것을 Linear interpolation이라고 한다.
* 이를 2차원으로 확장해서 4개의 점 사이에서 어떤 점의 값을 추정하는 것을 Bilinear interpolation이라고 한다.

<img src="./image/fcn3.png" />

FCN-32s의 경우, Upsampling만 하면 원하는 세그멘테이션 맵을 얻을 수 있다.

하지만 논문에서는 더 나은 성능을 위해서 한 가지 기법을 더해준다. 위 그림에서 확인할 수 있는 __Skip Architecture__라는 방법이다.

* FCN-16s는 앞쪽 블록에서 얻은 예측 결과 맵과, 2배로 upsampling한 맵을 더한 후, 한 번에 16배로 upsampling을 해주어 얻는다. 
* 한 번 더 앞쪽 블록을 사용하면 FCN-8s를 얻을 수 있다.

<img src="./image/fcn4.png" />

## 4. 주요 세그멘테이션 모델 (2) U-Net

> __*주요 참고 자료*__
>
> * *[딥러닝논문읽기모임의 U-Net: Convolutional Networks for Biomedical Image Segmentation](https://www.youtube.com/watch?v=evPZI9B2LvQ)*
> * *[U-Net 논문 리뷰 — U-Net: Convolutional Networks for Biomedical Image Segmentation](https://medium.com/@msmapark2/u-net-%EB%85%BC%EB%AC%B8-%EB%A6%AC%EB%B7%B0-u-net-convolutional-networks-for-biomedical-image-segmentation-456d6901b28a)*
> * *원본 논문: [U-Net: Convolutional Networks for Biomedical Image Segmentation](https://arxiv.org/pdf/1505.04597.pdf)*

<img src="./image/u_net.png" />
<center><b>https://arxiv.org/pdf/1505.04597.pdf</b></center>

U-Net은 이름에서 볼 수 있듯이 네트워크 구조가 U자 형태를 띠고 있다. __FCN에서 upsampling을 통해서 특성 맵을 키운 것을 입력값과 대칭적으로 만들어 준 것__이다.

### 전체 구조
---
논문에서는 네트워크 구조를 좌측의 Contracting path와 우측의 Expansive path 두 가지로 구분한다.

* 우측의 Contracting path는 일반적으로 우리가 사용해왔던 Convolution network와 유사한 구조를 가진다. 각 블록은 두개의 3x3 convolution 계층과 ReLu를 가지고 그 뒤로 downsampling을 위해서 2x2의 커널을 2 stride로 max pooling을 하게 된다. Downsampling을 거친 후 다음 convolution의 채널 크기는 두 배씩 늘어나도록 설계되었다.

* Expansive path에서는 각 블록에 2x2 up-convolution이 붙어 채널이 절반씩 줄어들고 특성 맵의 크기는 늘어난다. Expansive path의 블록은 contracting block과 동일하게 3x3 convolution이 두 개씩 사용되었다.

* 두 Path에서 크기가 같은 블록의 출력과 입력은 __skip connection처럼 연결해주어 low-level의 feature를 활용__할 수 있도록 하였다. 마지막에는 1x1 convolution으로 원하는 시맨틱 세그멘테이션 맵을 얻을 수 있다.

결과적으로는, 입력으로 572x572 크기인 이미지가 들어가고 출력으로 388x388의 크기에 두 가지의 클래스를 가진 세그멘테이션 맵(segmentation map)이 나온다.

마지막 세그멘테이션 맵의 크기가 입력 이미지와 다른 것은 앞에서 이야기한 것처럼 세그멘테이션 맵을 원하는 크기로 조정하여(resize) 해결할 수 있다.

### 타일(Tile) 기법
---
<img src="./image/tile.png" />

* FCN은 입력 이미지의 크기를 조정하여 세그멘테이션 맵을 얻어냈다. 
* U-Net은 타일(tile) 방식을 사용해서 어느 정도 서로 겹치는 구간으로 타일을 나누어 네트워크를 추론, 큰 이미지에서도 높은 해상도의 세그멘테이션 맵을 얻을 수 있도록 했다.

### 데이터 불균형 해결
---
<img src="./image/u_net2.png" />

세포를 검출해 내기 위해서는 세포들의 영역뿐만 아니라 경계 또한 예측을 해야 한다. 이때 픽셀 단위로 라벨을 매긴다고 생각하면, 데이터셋에 세포나 배경보다는 절대적으로 세포 간 경계의 면적이 작을 것이다. 이러한 __클래스 간 데이터 양의 불균형을 해결__해 주기 위해서 분포를 고려한 __weight map__을 학습 때 사용했다고 한다.

여기서 weight map의 weight를 신경망의 학습 파라미터를 가리키는 weight라고 착각할 수도 있을 것 같다. 실제로 여기서 말하는 weight는 __손실함수(loss)에 적용되는 가중치__를 말한다. 의료 영상에서 세포 내부나 배경보다는 상대적으로 면적이 작은 세포 경계를 명확하게 추론해 내는 것이 더욱 중요하기 때문에, 세포 경계의 손실에 더 많은 페널티를 부과하는 방식이다.

## 5. 주요 세그멘테이션 모델 (3) DeepLab 계열

> __*주요 참고 자료*__
>
> * _[Lunit 기술블로그의 DeepLab V3+: Encoder-Decoder with Atrous Separable Convolution for Semantic Image Segmentation](https://blog.lunit.io/2018/07/02/deeplab-v3-encoder-decoder-with-atrous-separable-convolution-for-semantic-image-segmentation/)_
> * _[hyunjulie님의 2편: 두 접근의 접점, DeepLab V3+](https://medium.com/hyunjulie/2%ED%8E%B8-%EB%91%90-%EC%A0%91%EA%B7%BC%EC%9D%98-%EC%A0%91%EC%A0%90-deeplab-v3-ef7316d4209d)_
> * _[Taeoh Kim님의 PR-045: DeepLab: Semantic Image Segmentation](https://www.youtube.com/watch?v=JiC78rUF4iI&ab_channel=TaeohKim)_
> * _원본 논문: [Encoder-Decoder with Atrous Separable Convolution for Semantic Image Segmentation](https://arxiv.org/pdf/1802.02611.pdf)_

* 처음 DeepLab 모델이 제안된 뒤 이 모델을 개선하기 위해 Atrous Convolution와 Spatial Pyramid Pooling 등 많은 방법들이 제안되어 왔다. 

### 전체 구조
---

<img src="./image/deeplab_v3.png" />

U-Net에서의 Contracting path과 Expansive path의 역할을 하는 것이 여기서는 위 그림의 인코더(Encoder), 디코더(Decoder)이다.

* 인코더는 이미지에서 필요한 정보를 특성으로 추출해내는 모듈이다.
* 디코더는 추출된 특성을 이용해 원하는 정보를 예측하는 모듈이다. 

3x3 convolution을 사용했던 U-Net과 달리 DeepLabV3+는 Atrous Convolution을 사용하고 있다. Atrous Convolution을 여러 크기에 다양하게 적용한 것이 ASPP(Atrous Spatial Pyramid Pooling)이다. DeepLab V3+는 ASPP가 있는 블록을 통해 특성을 추출하고 디코더에서 Upsampling을 통해 세그멘테이션 마스크를 얻는다.

### Atrous Convolution
---

<img src="./image/atrous_conv.gif" />
<img src="./image/atrous_conv2.gif" />

Atrous Convolution은 간단히 말하면 "띄엄띄엄 보는 컨볼루션"이다. 이를 통해 컨볼루션 레이어를 너무 깊게 쌓지 않아도 넓은 영역의 정보를 커버할 수 있게 된다.

* 참고: [딥러닝에서 사용되는 여러 유형의 Convolution 소개](https://zzsza.github.io/data/2018/02/23/introduction-convolution/)

### Spatial Pyramid Pooling
---
<img src="./image/SPP.png" />
<center><b>[[출처] http://kaiminghe.com/eccv14sppnet/index.html]</b></center>

* Spatial Pyramid Pooling은 여러 가지 스케일로 convolution과 pooling을 하고 나온 다양한 특성을 연결(concatenate)해준다. 
* 이를 통해서 멀티스케일로 특성을 추출하는 것을 병렬로 수행하는 효과를 얻을 수 있다. 
* 여기서 컨볼루션을 Atrous Convolution으로 바꾸어 적용한 것은 Atrous Spatial Pyramid Pooling이라고 한다. 

이러한 아키텍쳐는 입력이미지의 크기와 관계없이 동일한 구조를 활용할 수 있다는 장점이 있다. 그러므로 제각기 다양한 크기와 비율을 가진 RoI 영역에 대해 적용하기에 유리하다.

* 참고: [갈아먹는 Object Detection - Spatial Pyramid Pooling Network](https://yeomko.tistory.com/14)